In [ ]:
import re
from pathlib import Path
from typing import Dict

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:

def read_mnist_log_file(path: Path):
    lines = path.read_text().splitlines()
    progress_lines = [l for l in lines if "fit progress: (" in l]
    entries = []
    first_round_start_line = [l for l in progress_lines if "fit_round: strategy sampled" in l][0]
    first_round_start_time_secs = pd.to_datetime(" ".join(first_round_start_line.split(" ")[2:4])).total_seconds()
    for l in progress_lines:
        x = l.split("fit progress: ")[-1]
        round, loss, metrics, time = eval(x)
        entries.append(
            {
                "round": round,
                "loss": loss,
                "metrics": metrics,
                "accuracy": metrics.get("accuracy"),
                "time_since_start": time,
                "time": (time - first_round_start_time_secs) if len(entries) == 0 else (time - entries[-1]["time_since_start"])
            }
        )
    times_agg_eval = []
    for idx, line in enumerate(lines):
        if idx + 1 == len(lines):
            break
        next_line = lines[idx + 1]
        if "fit_round received" not in line:
            continue
        if "fit progress: (" in next_line:
            t_start = pd.to_datetime(" ".join(line.split(" ")[2:4]))
            t_end = pd.to_datetime(" ".join(next_line.split(" ")[2:4]))
            times_agg_eval.append((t_end - t_start).total_seconds())
    #"time_agg_eval": time_agg_eval,
    df = pd.DataFrame.from_records(entries)
    df["time_agg_eval"] = times_agg_eval
    new_dtypes = {"time": float, "loss": float, "accuracy": float,
                  "time_since_start": float, "round": int,
                  "time_agg_eval": float
                  }
    if not df.empty:
        df = df.astype(new_dtypes)
    return df


def read_leaf_log_file(f_err: Path, f_out: Path):
    out_lines = f_out.read_text().splitlines()

    # Metrics
    eval_entries = []
    for idx, l in enumerate(out_lines):
        matches = re.findall(r"EvaluateRes([^\)]+)", l)
        client_accs = []
        client_cardinalities = []
        client_losses = []
        for m in matches:
            eval_dict = eval(f"dict{m})")
            client_acc = eval_dict.get("metrics").get("accuracy")
            client_cardinality = eval_dict.get("num_examples")
            client_loss = eval_dict.get("loss")
            client_accs.append(client_acc)
            client_cardinalities.append(client_cardinality)
            client_losses.append(client_loss)
        if len(matches) == 0:
            continue
        acc = np.average(client_accs, weights=client_cardinalities)
        loss = np.average(client_losses, weights=client_cardinalities)
        eval_entries.append({
            "round": idx + 1,
            "accuracy": acc,
            "loss": loss
        })
    eval_entries = eval_entries[:-1]
    df = pd.DataFrame.from_records(eval_entries)
    if not df.empty:
        df = df.astype({"round": int, "accuracy": float, "loss": float})

    # Timing info
    err_lines = f_err.read_text().splitlines()
    timing_entries = []
    t_start_training = None
    for idx, line in enumerate(err_lines):
        if "fit_round: strategy sampled" not in line:
            continue
        try:
            received_line = err_lines[idx + 1]
            eval_start_line = err_lines[idx + 2]
            eval_end_line = err_lines[idx + 3]
            assert "evaluate_round received" in eval_end_line
            assert "evaluate_round: strategy sampled" in eval_start_line
            assert "fit_round received" in received_line
        except (IndexError, AssertionError) as e:
            continue
        t_fit_start = pd.to_datetime(" ".join(line.split(" ")[2:4]))
        t_fit_end = pd.to_datetime(" ".join(received_line.split(" ")[2:4]))
        t_eval_start = pd.to_datetime(" ".join(eval_start_line.split(" ")[2:4]))
        t_eval_end = pd.to_datetime(" ".join(eval_end_line.split(" ")[2:4]))
        if not t_start_training:
            t_start_training = t_fit_start
        total_time = (t_eval_end - t_fit_start).total_seconds()
        timing_entries.append({
            "time": total_time,
            "time_eval": (t_eval_end - t_eval_start).total_seconds(),
            "time_agg_eval": (t_eval_end - t_fit_end).total_seconds(),
            "time_clients_fit": (t_fit_end - t_fit_start).total_seconds(),
            "time_since_start": (t_eval_end - t_start_training).total_seconds()
        })
        #total_seconds
#    assert len(timing_entries) == len(df)
    timing_df = pd.DataFrame.from_records(timing_entries[:len(df)])

    return df.join(timing_df)

In [ ]:
files = []
dfs = []
LOG_FOLDER = Path("/Users/andreas/workspace/thesis-code/out/flower-logs")
for f in LOG_FOLDER.glob("fedless_*.err"):
    if (len(f.name.split("_"))) == 6:  # Local Client Log
        _, dataset, clients_in_round, clients_total, local_epochs, seed = f.name.split("_")
        batch_size = 5
    elif (len(f.name.split("_"))) == 7:  # Local Client Log
        _, dataset, clients_in_round, clients_total, local_epochs, batch_size, seed = f.name.split("_")
    else:
        continue
    seed = seed.split(".")[0]
    if dataset == "mnist":  # All required data lies in .err file
        logs_df = read_mnist_log_file(f)
    elif dataset in ["femnist", "shakespeare"]:
        logs_df = read_leaf_log_file(f_err=f, f_out=f.with_suffix(".out"))

    if logs_df.empty:
        continue

    index = pd.MultiIndex.from_tuples(
        [(
            dataset,
            clients_in_round,
            clients_total,
            local_epochs,
            batch_size,
            seed
        )] * len(logs_df),
        names=[
            "dataset",
            "clients_in_round",
            "clients_total",
            "local_epochs",
            "batch_size",
            "seed"
        ]
    )
    df = pd.DataFrame(logs_df.values, index=index, columns=logs_df.columns)  # .reset_index()
    df = df.astype(logs_df.dtypes)

    integer_index_levels = [1, 2, 3]
    for i in integer_index_levels:
        df.index = df.index.set_levels(df.index.levels[i].astype(int), level=i)
    dfs.append(df)
dfs = pd.concat(dfs)
dfs = dfs.sort_index()

In [ ]:
#dfs.loc[{"seed": 25738}]
#mnist_dfs = dfs.loc[("mnist", "100", "100", "10")]
mnist_dfs = dfs.loc[("mnist", 75, 100, 5)]
mnist_dfs[mnist_dfs["accuracy"] >= 0.99].groupby("seed").min("round")["round"].mean()
#sns.lineplot(x="round", y="accuracy", data=mnist_dfs)

In [ ]:
df_ = dfs.loc[("mnist", slice(75, 100), 100, slice(1, 100)), :]
(
    df_[df_["accuracy"] >= 0.99].groupby(by=["seed", "clients_in_round", "local_epochs"]).min("round")
        .groupby(["clients_in_round", "local_epochs"])
        .mean()
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))
df_ = dfs.loc[("mnist", slice(75, 100), 100, slice(1, 100)), :]
df_ = df_[df_["accuracy"] >= 0.99].groupby(level=[1, 2, 3, 4]).min("round")
df_ = df_.groupby(level=[0, 2]).mean()
sns.barplot(y="round", x="local_epochs", data=df_.reset_index(), ax=ax1)  # hue="local_epochs",

#df_ = dfs.loc[("mnist", 100, 100, slice(1, 100)), :]
#df_ = df_[df_["accuracy"] >= 0.98].groupby(level=[1, 2, 3, 4]).min("round")
#df_ = df_.groupby(level=[0, 2]).mean()
sns.barplot(y="time_since_start", x="local_epochs", data=df_.reset_index(), ax=ax2)  # hue="local_epochs",
fig.set_label(f"Test123")
fig.savefig("/Users/andreas/Desktop/comp.pdf")
df_

In [ ]:
fig, axarr = plt.subplots(ncols=2, figsize=(10, 5))
df_ = dfs.loc[("mnist", 75, 100, slice(1, 100)), :]
df_ = df_[df_["round"] > 1]
#sns.barplot(y="time", x="local_epochs", data=df_.reset_index())  #  hue="local_epochs"
sns.lineplot(y="time", x="round", data=df_.reset_index(), hue="local_epochs", ax=axarr[0])
sns.boxplot(y="time", x="local_epochs", data=df_.reset_index(), ax=axarr[1])

In [ ]:
df_ = dfs.loc[("femnist", 75, 100, slice(1, 100)), :]
df_ = df_[df_["round"] > 1]
df_["time_wo_agg_eval"] = df_["time"] - df_["time_agg_eval"]
df_[["time_wo_agg_eval", "time_agg_eval"]].mean(level=[3]).plot.bar(stacked=True)
df_.mean(level=[3])

In [ ]:
femnist_dfs = dfs.loc[("femnist", 100, 100, slice(1, 100)), :]
femnist_dfs[femnist_dfs["accuracy"] >= 0.8].groupby("seed").min("round")["round"].mean()

In [ ]:
df_ = dfs.loc[("femnist", 100, 100, slice(1, 100)), :]
df_ = df_[df_["accuracy"] >= 0.80].groupby(level=[1, 2, 3, 4]).min("round")
df_ = df_.groupby(level=[0, 2]).mean()
#sns.barplot(y="round", x="local_epochs", data=df_.reset_index()) # hue="local_epochs",
df_ = dfs.loc[("femnist", 100, 100, slice(1, 100)), :]
sns.lineplot(y="accuracy", x="round", data=df_.reset_index(), hue="local_epochs")  # hue="local_epochs",

In [ ]:
df_ = dfs.loc[("shakespeare", slice(1, 100), slice(1, 100), slice(1, 100)), :]
df_ = df_[df_["round"] > 1]
df_.groupby("seed").max()

In [ ]:
df_ = dfs.loc[("shakespeare", slice(1, 100), slice(1, 100), slice(1, 100)), :]
sns.lineplot(y="accuracy", x="round", data=df_.reset_index(), hue="batch_size")  # hue="local_epochs",
#sns.lineplot(y="time_since_start", x="round", data=df_.reset_index(), hue="batch_size")  # hue="local_epochs",
#sns.lineplot(y="time", x="round", data=df_.reset_index(), hue="local_epochs")  # hue="local_epochs",

In [ ]:
dfs